In [9]:
#We can go into our root file and see what Trees are availiable
%matplotlib inline
import sys, os
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
    sys.path.append(os.path.realpath("../../"))
import numpy as np
import pandas as pd
import ntpath
import glob
from CMS_SURF_2016.utils.metrics import plot_history, print_accuracy_m
from CMS_SURF_2016.utils.callbacks import OverfitStopping

from keras.models import Sequential
from keras.layers import Dense, Flatten, Reshape, Activation, Dropout, Convolution2D
from keras.callbacks import EarlyStopping

#The observables taken from the table
observ_types = ['E/c', 'Px', 'Py', 'Pz', 'PID', 'Charge']
set_size = 20
vecsize = len(observ_types)
epochs = 100
batch_size = 100

#http://pdg.lbl.gov/2012/reviews/rpp2012-rev-monte-carlo-numbering.pdf
observable_vocab = [0,-11,11,15,-15,-13,13,22]
quark_vocab = [5,-5,6,-6]
neutrino_vocab = [12,-12, 14, -14, 16, -16, 18, -18]
unobservable_boson_vocab = [24, -24]

particle_vocab = observable_vocab + neutrino_vocab + quark_vocab +  unobservable_boson_vocab
particle_dict = {particle_vocab[i]:i for i in range(len(particle_vocab))}
#print(particle_dict)
def cullNonObservables(frame):
    #Status of 1 means that the particle is a stable product
    stable_cond = frame["Status"] == 1 
    #All even leptons are neutrinos which we can't measure
    notNeutrino_cond = np.abs(frame["PID"] % 2) == 1
    #Get all entries that satisfy the conditions
    frame = frame[stable_cond & notNeutrino_cond]
    #Drop the Status frame since we only needed it to see if the particle was stable
    frame = frame.drop(["Status"], axis=1)
    return frame

def mapPIDS(observables):
    PIDS = observables["PID"] 
    observables["PID"] = PIDS.apply(lambda x: particle_dict[x])

def padItem(x, shuffle=False):
    if(len(x) > set_size):
        return x[:set_size]
    else:
        out = np.append(x ,np.array(np.zeros((set_size - len(x), vecsize))), axis=0)
        if(shuffle): np.random.shuffle(out)
        return out
def padInput(l,shuffle=False):
    out = []
    for x in l:
        out.append(padItem(x, shuffle=shuffle))
    return out

def helper_gETA(x, arr):
    arr.append(np.array(x))
    return 0
def groupEntriesToArrays(frame, select):
    arr = []
    grouped = frame.groupby(["Entry"])[select].apply(lambda x: helper_gETA(x,arr))
    return arr
def preprocessFromPandas_file_label_pairs(files, cull=False):
    X_train = []
    y_train = []
    X_train_by_label = {}
    y_train_by_label = {}
    for f,label in files:
        all_particles = pd.read_hdf(f, 'data')
        if(cull):
            observables = cullNonObservables(all_particles)
        else:
            observables = all_particles
        
        mapPIDS(observables)
        processedInput = padInput(groupEntriesToArrays(observables, observ_types))
       
        X_train_by_label[label] = X_train_by_label.get(label, []) + processedInput
        y_train_by_label[label] = y_train_by_label.get(label, []) + ([label] * len(processedInput))
    
    #Truncate the data so that we have the same amount in each catagory
    minimumN = min([len(X_train_by_label[label]) for label in X_train_by_label])
    for label in X_train_by_label:
        X_train = X_train + X_train_by_label[label][:minimumN]
        y_train = y_train + y_train_by_label[label][:minimumN]
    X_train_by_label = None
    y_train_by_label = None
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    indices = np.arange(len(y_train))
    np.random.shuffle(indices)
    X_train = X_train[indices]
    y_train = y_train[indices]
   
    return X_train, y_train



In [ ]:
nFiles = 10
ttbar_files = glob.glob("/data/shared/Delphes/ttbar_13TeV/parsed_pandas/*.h5")
WJet_files = glob.glob("/data/shared/Delphes/WJets_13TeV/parsed_pandas/*.h5")
files = []
for i in range(nFiles):
    files.append((ttbar_files[i],0))
    files.append((WJet_files[i],1))



In [ ]:
histories = {}
cull=False
add_title = "_AllParticles"
for i in range (2):
   
    X_train, y_train = preprocessFromPandas_file_label_pairs(files, cull=cull)
    X_train_flatten = np.array([np.ndarray.flatten(x) for x in X_train])
    #DENSE
    dense = Sequential()
    dense.add(Dense(10, input_dim=set_size * vecsize,activation='relu'))
    #model.add(Dropout(.5))
    dense.add(Dense(1, activation='sigmoid'))
    dense.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    #CONVOLUTIONAL
    conv = Sequential()
    conv.add(Convolution2D(40,4,4, input_shape=(1,set_size,vecsize),activation='relu'))
    conv.add(Flatten())
    conv.add(Dense(1, activation='sigmoid'))
    conv.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(verbose=1, patience=10)
    overfitStopping = OverfitStopping(verbose=1, patience=10)
    #RUN Dense
    dense_history = dense.fit(X_train_flatten, y_train,
                        batch_size=batch_size,
                        nb_epoch=epochs,
                        validation_split=.2,
                        callbacks=[earlyStopping, overfitStopping])
    histories["dense"+add_title] = (dense,dense_history,X_train_flatten, y_train)
#   plot_history([("dense",dense_history)])

    
    earlyStopping = EarlyStopping(verbose=1, patience=10)
    overfitStopping = OverfitStopping(verbose=1, patience=10)
    #Run Conv
    conv_history = conv.fit(np.reshape(X_train, (len(X_train), 1, set_size, vecsize)), y_train,
                        batch_size=batch_size,
                        nb_epoch=epochs,
                        validation_split=.2,
                        callbacks=[earlyStopping, overfitStopping])
    #plot_history([("conv",conv_history)])
    histories["conv"+add_title] = (conv,conv_history,X_train, y_train)
    add_title = "_ObservableOnly"
    cull=True

Train on 143080 samples, validate on 35770 samples
Epoch 1/100
143080/143080 [==============================] - 2s - loss: 0.9967 - acc: 0.9291 - val_loss: 0.2404 - val_acc: 0.9829
Epoch 2/100
121600/143080 [========================>.....] - ETA: 0s - loss: 0.2281 - acc: 0.9837

In [ ]:
keys = [key for key in histories]
def p(key):
    tup = histories[key]
    model = tup[0]
    history = tup[1]
    #print_accuracy_m(model, tup[2], tup[3])
    print(key + ': Best Validation accuracy: %r%%' % max(history.history["val_acc"]))
    plot_history([(key, history)])


In [ ]:
p(keys[0])

In [ ]:
p(keys[1])

In [ ]:
p(keys[2])

In [ ]:
p(keys[3])